In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision import models, transforms
from transformers import CLIPProcessor, CLIPModel
from PIL import Image
import os
import pandas as pd
import json
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from google.colab import drive

# JSON configuration
config_json = """
{
    "device": "cuda",
    "dataset_path": "/content/drive/MyDrive/archive/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)/train",
    "disease_directories": [
  "Apple___Apple_scab",
  "Apple___Black_rot",
  "Apple___Cedar_apple_rust",
  "Blueberry___healthy",
  "Cherry_(including_sour)___Powdery_mildew",
  "Corn_(maize)___Cercospora_leaf_spot Gray_leaf_spot",
  "Corn_(maize)___Common_rust_",
  "Corn_(maize)___Northern_Leaf_Blight",
  "Grape___Black_rot",
  "Grape___Esca_(Black_Measles)",
  "Grape___Leaf_blight_(Isariopsis_Leaf_Spot)",
  "Orange___Haunglongbing_(Citrus_greening)",
  "Peach___Bacterial_spot",
  "Pepper,_bell___Bacterial_spot",
  "Potato___Early_blight",
  "Potato___Late_blight",
  "Raspberry___healthy",
  "Soybean___healthy",
  "Squash___Powdery_mildew",
  "Strawberry___Leaf_scorch",
  "Tomato___Spider_mites Two-spotted_spider_mite",
  "Tomato___Tomato_Yellow_Leaf_Curl_Virus",
  "Tomato___Tomato_mosaic_virus"
]
,
    "annotations": [
  {
    "filename": "Apple___Apple_scab",
    "disease": "Apple Scab",
    "symptoms": "Circular, olive-green to dark brown lesions on leaves, fruit, and young twigs. Leaves may become distorted, curl, or drop prematurely. Infected fruit develops rough, scaly patches and may crack. Severe infections can cause defoliation, reducing fruit yield and quality.",
    "treatment": "Apply fungicides like captan, mancozeb, or myclobutanil early in the season, especially before rainy periods. Use resistant apple varieties such as 'Enterprise', 'Liberty', and 'Freedom'. Remove fallen leaves and infected fruit to prevent overwintering of the fungus. Prune trees to improve air circulation and reduce moisture buildup. Avoid overhead watering and ensure proper spacing between trees for better air circulation."
  },
  {
    "filename": "Apple___Black_rot",
    "disease": "Black Rot (Apple)",
    "symptoms": "Small, circular brown spots on leaves that enlarge and develop a reddish-purple border. Infected fruit shows concentric rings, turns black, and shrivels into mummified fruit. Cankers form on branches, appearing as sunken, dark lesions with cracked bark.",
    "treatment": "Prune and destroy infected branches and mummified fruit to prevent the spread of the fungus. Apply copper-based or sulfur fungicides during early bloom and after petal fall. Ensure good air circulation by proper pruning and spacing of trees. Avoid overhead watering to reduce moisture levels. Maintain orchard sanitation by removing fallen leaves and fruit."
  },
  {
    "filename": "Apple___Cedar_apple_rust",
    "disease": "Cedar Apple Rust",
    "symptoms": "Bright orange, gelatinous spore masses appear on the undersides of leaves. Small yellow spots develop on the upper leaf surface, expanding into orange-red lesions. Infected fruit may show similar spots and become deformed. Severe infections can cause early leaf drop and reduced fruit yield.",
    "treatment": "Remove or prune nearby cedar and juniper trees to break the disease cycle. Apply protective fungicides like myclobutanil, mancozeb, or propiconazole during early spring and throughout the growing season. Plant resistant apple varieties such as 'Liberty' and 'Enterprise'. Ensure good air circulation by proper pruning and avoid overhead watering to reduce leaf wetness."
}
,
  {
    "filename": "Blueberry___healthy",
    "disease": "Healthy (Blueberry)",
    "symptoms": "No disease symptoms. Leaves are green and firm, with no discoloration, spots, or wilting. Stems are strong, and fruit develops normally without signs of rot or mold.",
    "treatment": "Maintain good agricultural practices, including proper irrigation, balanced fertilization, and regular pruning to promote air circulation. Monitor plants for early signs of disease or pest infestations. Ensure well-drained soil with appropriate pH (4.5–5.5) to support healthy growth. Mulch around plants to retain moisture and suppress weeds."
}
,
  {
    "filename": "Cherry_(including_sour)___Powdery_mildew",
    "disease": "Powdery Mildew (Cherry)",
    "symptoms": "White, powdery fungal growth appears on the upper and lower leaf surfaces. Leaves may curl, become distorted, and show yellowing. In severe cases, buds, young shoots, and fruit can also be affected, leading to stunted growth and reduced yield.",
    "treatment": "Apply sulfur-based fungicides or potassium bicarbonate sprays at the first sign of infection. Prune affected branches and remove fallen leaves to prevent the spread of spores. Ensure proper spacing and pruning to improve air circulation. Avoid excessive nitrogen fertilization, which can encourage mildew growth. Water plants at the base to keep foliage dry and reduce humidity levels."
}
,
  {
    "filename": "Corn_(maize)___Cercospora_leaf_spot_Gray_leaf_spot",
    "disease": "Cercospora Leaf Spot (Corn)",
    "symptoms": "Small, grayish-brown rectangular lesions with purple to reddish-brown borders appear on lower leaves first. Lesions may enlarge and merge, leading to extensive leaf blight. Severe infections reduce photosynthesis, causing premature leaf death and lower yield.",
    "treatment": "Rotate crops with non-host plants like soybeans or wheat to reduce fungal spore buildup. Use resistant or tolerant corn varieties for better disease management. Apply fungicides such as strobilurins or triazoles during early infection stages. Improve field drainage and ensure proper plant spacing to reduce humidity and promote airflow. Remove crop residues after harvest to minimize fungal overwintering."
}
,
  {
    "filename": "Corn_(maize)___Common_rust_",
    "disease": "Common Rust (Corn)",
    "symptoms": "Small, reddish-brown pustules appear on both leaf surfaces, often surrounded by a yellow halo. As the disease progresses, pustules darken and merge, leading to leaf blight and reduced photosynthesis. Severe infections can weaken plants and lower yield.",
    "treatment": "Plant resistant corn hybrids to minimize infection risk. Apply fungicides such as strobilurins or triazoles when disease pressure is high. Ensure proper plant spacing and improve air circulation to reduce moisture buildup. Avoid excessive nitrogen fertilization, which can make plants more susceptible."
}
,
  {
    "filename": "Corn_(maize)___Northern_Leaf_Blight",
    "disease": "Northern Leaf Blight (Corn)",
    "symptoms": "Long, cigar-shaped gray-green lesions develop on lower leaves first and expand as the disease progresses. Lesions turn tan and may develop dark fungal spores under humid conditions. Severe infections cause leaf blight, reducing grain fill and yield.",
    "treatment": "Use resistant corn hybrids to prevent infections. Apply fungicides like azoxystrobin or propiconazole during early symptom development. Rotate crops with non-host plants to break the disease cycle. Improve field drainage and plant spacing to reduce humidity and promote air circulation."
}
,
  {
    "filename": "Grape___Black_rot",
    "disease": "Black Rot (Grape)",
    "symptoms": "Small, circular brown spots with darker borders appear on leaves, eventually enlarging and turning necrotic. Infected fruit develops sunken, dark brown lesions before shriveling into hard, black mummies. The disease spreads rapidly in warm, humid conditions.",
    "treatment": "Prune and destroy infected vines, leaves, and fruit to prevent fungal spread. Apply fungicides such as myclobutanil, mancozeb, or captan during early growth stages. Ensure proper air circulation by pruning and trellising. Avoid overhead watering and remove mummified berries to reduce fungal overwintering."
}
,
  {
    "filename": "Grape___Esca_(Black_Measles)",
    "disease": "Esca (Black Measles, Grape)",
    "symptoms": "Dark, irregular streaks appear on the woody tissues of vines. Leaves show interveinal necrosis with a tiger-stripe pattern of yellow and brown areas. Affected fruit may develop sunken, discolored spots and shrivel prematurely. Severe cases lead to vine dieback.",
    "treatment": "Avoid wounding vines during pruning to prevent fungal entry. Remove and destroy infected wood and prune symptomatic areas. Maintain vine health with balanced fertilization and proper irrigation. Improve vineyard drainage and minimize stress factors to reduce disease susceptibility."
}
,
  {
    "filename": "Grape___Leaf_blight_(Isariopsis_Leaf_Spot)",
    "disease": "Leaf Blight (Grape)",
    "symptoms": "Irregular brown lesions develop on leaves, often with a yellow halo. Severely infected leaves curl, dry out, and drop prematurely. The disease weakens vines and reduces fruit production.",
    "treatment": "Prune and remove affected leaves and branches to improve air circulation. Apply fungicides such as mancozeb or copper-based sprays during early infection stages. Ensure proper vineyard sanitation and avoid overhead irrigation to minimize moisture on leaves."
}
,
  {
    "filename": "Orange___Haunglongbing_(Citrus_greening)",
    "disease": "Citrus Greening (Huanglongbing)",
    "symptoms": "Leaves exhibit asymmetrical yellowing with blotchy mottling. Fruits remain small, misshapen, and have a bitter taste. Infected trees show stunted growth, twig dieback, and premature fruit drop.",
    "treatment": "Control psyllid insect vectors using insecticides and biological control methods. Remove and destroy infected trees to prevent disease spread. Use disease-free nursery stock for new plantings and implement strict quarantine measures in affected areas."
}
,
  {
    "filename": "Peach___Bacterial_spot",
    "disease": "Bacterial Spot (Peach)",
    "symptoms": "Small, dark, water-soaked lesions form on leaves, which later turn reddish-brown with a shot-hole appearance. Fruit develops sunken, dark spots that may ooze and crack, reducing market quality.",
    "treatment": "Apply copper-based sprays or bactericides like streptomycin during early growth stages. Avoid overhead irrigation to reduce bacterial spread. Plant resistant peach varieties and practice crop rotation to minimize recurrence."
}
,
  {
    "filename": "Pepper,_bell___Bacterial_spot",
    "disease": "Bacterial Spot (Pepper)",
    "symptoms": "Water-soaked lesions appear on leaves, enlarging into dark brown spots with yellow halos. Severe infections cause leaf drop, stunted growth, and blemished fruit, reducing yield.",
    "treatment": "Use certified disease-free seeds and resistant varieties. Apply copper-based bactericides to manage early infections. Avoid working in wet fields to prevent bacterial spread and use proper crop rotation techniques."
}
,
  {
    "filename": "Potato___Early_blight",
    "disease": "Early Blight (Potato)",
    "symptoms": "Dark, concentric rings form on older leaves, resembling a target pattern. Leaves turn yellow and wither as the disease progresses. Severe infections reduce tuber size and yield.",
    "treatment": "Plant resistant potato varieties and ensure proper crop rotation. Apply fungicides such as chlorothalonil or azoxystrobin at the first sign of infection. Maintain good field sanitation and remove infected plant debris after harvest."
}
,
  {
    "filename": "Potato___Late_blight",
    "disease": "Late Blight (Potato)",
    "symptoms": "Dark, water-soaked lesions appear on leaves, rapidly expanding under humid conditions. A white, fuzzy mold develops on the undersides of infected leaves. Stems and tubers may also show dark, rotting areas, leading to plant collapse and severe yield loss.",
    "treatment": "Apply fungicides like chlorothalonil, mancozeb, or metalaxyl at the first sign of infection. Avoid overhead irrigation to reduce leaf wetness. Ensure proper field drainage and remove infected plants to prevent disease spread. Rotate crops and use disease-free seed potatoes."
}
,
  {
    "filename": "Raspberry___healthy",
    "disease": "Healthy (Raspberry)",
    "symptoms": "No disease symptoms. Leaves are green and firm, with no spots, wilting, or discoloration. Canes are strong and productive, producing healthy fruit.",
    "treatment": "Maintain proper crop management, including pruning for good air circulation. Monitor for pests and diseases regularly. Use well-drained soil with proper irrigation and mulching to retain moisture and suppress weeds. Fertilize appropriately for optimal growth."
}
,
  {
    "filename": "Soybean___healthy",
    "disease": "Healthy (Soybean)",
    "symptoms": "No disease symptoms. Leaves are green and fully developed without signs of spots, yellowing, or curling. Plants grow uniformly and produce healthy pods.",
    "treatment": "Use proper agricultural practices, including crop rotation, balanced fertilization, and timely irrigation. Monitor for pests and diseases regularly. Plant high-quality, disease-resistant soybean varieties to maintain healthy crops."
}
,
  {
    "filename": "Squash___Powdery_mildew",
    "disease": "Powdery Mildew (Squash)",
    "symptoms": "White, powdery fungal growth appears on the upper surfaces of leaves, stems, and sometimes fruit. Leaves may curl, yellow, and eventually dry out, reducing plant vigor and yield.",
    "treatment": "Apply sulfur-based fungicides or potassium bicarbonate sprays at the first sign of infection. Improve air circulation by properly spacing plants and pruning excess foliage. Avoid overhead watering and plant resistant squash varieties when available."
}
,
  {
    "filename": "Strawberry___Leaf_scorch",
    "disease": "Leaf Scorch (Strawberry)",
    "symptoms": "Small, reddish-brown spots with yellow halos appear on leaves. As the disease progresses, spots enlarge and merge, causing leaf edges to dry out and curl. Severe infections reduce plant vigor and fruit production.",
    "treatment": "Improve air circulation by proper plant spacing and pruning. Apply fungicides such as copper-based sprays or myclobutanil at early symptom stages. Water at the base of plants to keep foliage dry and reduce fungal spread. Remove and destroy infected leaves to limit disease progression."
}
,
  {
    "filename": "Tomato___Spider_mites_Two-spotted_spider_mite",
    "disease": "Spider Mites (Two-Spotted Spider Mite, Tomato)",
    "symptoms": "Tiny yellow stippling appears on leaves as mites feed on plant cells. Fine webbing is visible on the undersides of leaves and between stems. Severe infestations cause leaf bronzing, curling, and defoliation, leading to reduced plant vigor and yield.",
    "treatment": "Use miticides such as abamectin or spiromesifen for effective control. Introduce predatory mites like Phytoseiulus persimilis to naturally reduce populations. Spray plants with neem oil or insecticidal soap to suppress infestations. Maintain proper irrigation to reduce plant stress, which makes them more susceptible to mites."
}
,
  {
    "filename": "Tomato___Tomato_Yellow_Leaf_Curl_Virus",
    "disease": "Tomato Yellow Leaf Curl Virus",
    "symptoms": "Young leaves curl upwards, become thickened, and turn yellow. Plants exhibit stunted growth and reduced fruit production. Flowers may drop prematurely, leading to poor yields.",
    "treatment": "Control whiteflies, the primary vectors, using insecticides or neem oil. Introduce beneficial insects like ladybugs to reduce whitefly populations. Plant virus-resistant tomato varieties to minimize infection risk. Use physical barriers such as insect-proof netting to prevent whitefly infestation."
}
,
  {
    "filename": "Tomato___Tomato_mosaic_virus",
    "disease": "Tomato Mosaic Virus",
    "symptoms": "Leaves develop a mottled or mosaic pattern of light and dark green areas. Plants exhibit stunted growth, distorted leaves, and reduced fruit yield. Infected fruits may show discoloration or uneven ripening.",
    "treatment": "Use virus-free seeds and resistant tomato varieties. Remove and destroy infected plants to prevent the spread. Disinfect tools and hands after handling plants to reduce mechanical transmission. Control aphids and other potential insect vectors that may spread the virus."
}

]
,
    "train_batch_size": 28,
    "num_epochs": 20,
    "learning_rate": 0.001
}
"""

# Parse JSON configuration
config = json.loads(config_json)
device = config["device"] if torch.cuda.is_available() else "cpu"
dataset_path = config["dataset_path"]
disease_directories = config["disease_directories"]

# Create CSV from annotations
annotations_df = pd.DataFrame(config["annotations"])
annotations_df.to_csv("disease_annotations.csv", index=False)

# Create dictionary for symptoms and treatments
disease_info = {
    annotation["disease"]: {
        "symptoms": annotation["symptoms"],
        "treatment": annotation["treatment"]
    }
    for annotation in config["annotations"]
}

# Define custom dataset class
class PlantDiseaseDataset(Dataset):
    def __init__(self, root_dir, disease_dirs, transform=None):
        self.image_paths = []
        self.labels = []
        self.transform = transform
        for idx, disease_dir in enumerate(disease_dirs):
            disease_path = os.path.join(root_dir, disease_dir)
            for img_name in os.listdir(disease_path):
                img_path = os.path.join(disease_path, img_name)
                self.image_paths.append(img_path)
                self.labels.append(idx)

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        image = Image.open(self.image_paths[idx]).convert("RGB")
        label = self.labels[idx]
        if self.transform:
            image = self.transform(image)
        return image, label

# Image transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# Initialize dataset and DataLoader
dataset = PlantDiseaseDataset(dataset_path, disease_directories, transform=transform)
train_paths, val_paths = train_test_split(dataset.image_paths, test_size=0.2, random_state=42)
train_dataset = PlantDiseaseDataset(dataset_path, disease_directories, transform=transform)
val_dataset = PlantDiseaseDataset(dataset_path, disease_directories, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=config["train_batch_size"], shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=config["train_batch_size"], shuffle=False)

# Define a simple CNN model
class SimpleCNN(nn.Module):
    def __init__(self, num_classes):
        super(SimpleCNN, self).__init__()
        self.model = models.resnet18(pretrained=True)
        self.model.fc = nn.Linear(self.model.fc.in_features, num_classes)

    def forward(self, x):
        return self.model(x)

# Initialize model, criterion, and optimizer
num_classes = len(disease_directories)
cnn_model = SimpleCNN(num_classes).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(cnn_model.parameters(), lr=config["learning_rate"])

# Function to calculate accuracy
def calculate_accuracy(model, dataloader):
    model.eval()
    total = 0
    correct = 0
    with torch.no_grad():
        for images, labels in dataloader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    accuracy = correct / total * 100
    return accuracy

# Lists to store metrics
train_accuracies = []
val_accuracies = []
train_losses = []

# Mount Google Drive
drive.mount('/content/drive')

# Define the path where the model will be saved in Google Drive
model_save_path = '/content/drive/MyDrive/plant_disease_cnn_model.pth'

# Training loop
for epoch in range(config["num_epochs"]):
    cnn_model.train()
    epoch_loss = 0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = cnn_model(images)
        loss = criterion(outputs, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()

    epoch_loss /= len(train_loader)
    train_losses.append(epoch_loss)

    # Calculate training and validation accuracy
    train_accuracy = calculate_accuracy(cnn_model, train_loader)
    val_accuracy = calculate_accuracy(cnn_model, val_loader)

    # Append accuracies for graphing
    train_accuracies.append(train_accuracy)
    val_accuracies.append(val_accuracy)

    print(f"Epoch [{epoch+1}/{config['num_epochs']}], Loss: {epoch_loss:.4f}, "
          f"Training Accuracy: {train_accuracy:.2f}%, Validation Accuracy: {val_accuracy:.2f}%")
    # Save the trained model to Google Drive after each epoch
    torch.save(cnn_model.state_dict(), model_save_path)

# Load the saved model
loaded_model = SimpleCNN(num_classes).to(device)
loaded_model.load_state_dict(torch.load(model_save_path))
loaded_model.eval()
print("Model loaded successfully.")

# Calculate and display final accuracies
total_train_accuracy = sum(train_accuracies) / len(train_accuracies)
total_val_accuracy = sum(val_accuracies) / len(val_accuracies)
print(f"\nTotal Training Accuracy: {total_train_accuracy:.2f}%")
print(f"Total Validation Accuracy: {total_val_accuracy:.2f}%")

# Test accuracy
test_dataset = PlantDiseaseDataset(dataset_path, disease_directories, transform=transform)
test_loader = DataLoader(test_dataset, batch_size=config["train_batch_size"], shuffle=False)
test_accuracy = calculate_accuracy(cnn_model, test_loader)
print(f"Test Accuracy: {test_accuracy:.2f}%")

# Plot graphs for training accuracy, validation accuracy, and loss
plt.figure(figsize=(12, 5))

# Plot accuracy
plt.subplot(1, 2, 1)
plt.plot(range(1, config["num_epochs"] + 1), train_accuracies, label="Train Accuracy")
plt.plot(range(1, config["num_epochs"] + 1), val_accuracies, label="Validation Accuracy")
plt.title("Training and Validation Accuracy")
plt.xlabel("Epoch")
plt.ylabel("Accuracy (%)")
plt.legend()

# Plot loss
plt.subplot(1, 2, 2)
plt.plot(range(1, config["num_epochs"] + 1), train_losses, label="Loss")
plt.title("Training Loss")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.legend()

plt.tight_layout()
plt.show()


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 72.5MB/s]


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Mount Google Drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms, models
from transformers import CLIPProcessor, CLIPModel
from PIL import Image
import os
import json
import pandas as pd
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from google.colab import drive


# JSON configuration
config_json = """
{
    "device": "cuda",
    "dataset_path": "/content/drive/MyDrive/archive/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)/train",
    "train_batch_size": 28,
    "num_epochs": 20,
    "learning_rate": 0.001
}
"""

# Parse JSON configuration
config = json.loads(config_json)
device = config["device"] if torch.cuda.is_available() else "cpu"
dataset_path = config["dataset_path"]

# Define disease classes (folders inside dataset path)
disease_directories = sorted(os.listdir(dataset_path))
num_classes = len(disease_directories)

# Image transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# Load CLIP model & processor
clip_model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32").to(device)
clip_processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

# Define dataset class
class PlantDiseaseDataset(Dataset):
    def __init__(self, image_paths, labels, transform=None):
        self.image_paths = image_paths
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        image = Image.open(self.image_paths[idx]).convert("RGB")
        label = self.labels[idx]

        if self.transform:
            image = self.transform(image)

        return image, label

# Prepare dataset paths & labels
image_paths, labels = [], []
for idx, disease_dir in enumerate(disease_directories):
    disease_path = os.path.join(dataset_path, disease_dir)
    for img_name in os.listdir(disease_path):
        image_paths.append(os.path.join(disease_path, img_name))
        labels.append(idx)

# Split dataset into train (80%) and validation (20%)
train_paths, val_paths, train_labels, val_labels = train_test_split(image_paths, labels, test_size=0.2, random_state=42)

# Create dataset instances
train_dataset = PlantDiseaseDataset(train_paths, train_labels, transform=transform)
val_dataset = PlantDiseaseDataset(val_paths, val_labels, transform=transform)

# Initialize DataLoaders
train_loader = DataLoader(train_dataset, batch_size=config["train_batch_size"], shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=config["train_batch_size"], shuffle=False)

# Define a CNN model with CLIP integration
class CLIP_CNN_Model(nn.Module):
    def __init__(self, num_classes):
        super(CLIP_CNN_Model, self).__init__()
        self.cnn_model = models.resnet50(pretrained=True)
        self.cnn_model.fc = nn.Linear(self.cnn_model.fc.in_features, 512)  # Extract 512D features
        self.fc = nn.Linear(512, num_classes)  # Classification layer

    def forward(self, x):
        features = self.cnn_model(x)  # CNN feature extraction
        output = self.fc(features)  # Classification output
        return output

# Initialize model, criterion, and optimizer
model = CLIP_CNN_Model(num_classes).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=config["learning_rate"])

# Accuracy function
def calculate_accuracy(model, dataloader):
    model.eval()
    total, correct = 0, 0
    with torch.no_grad():
        for images, labels in dataloader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    return correct / total * 100

# Training loop
train_accuracies, val_accuracies, train_losses = [], [], []
model_save_path = "/content/drive/MyDrive/plant_disease_clip_cnn.pth"

for epoch in range(config["num_epochs"]):
    model.train()
    epoch_loss = 0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()

        outputs = model(images)  # CNN-based classification
        loss = criterion(outputs, labels)

        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()

    epoch_loss /= len(train_loader)
    train_losses.append(epoch_loss)

    # Calculate accuracy
    train_acc = calculate_accuracy(model, train_loader)
    val_acc = calculate_accuracy(model, val_loader)

    train_accuracies.append(train_acc)
    val_accuracies.append(val_acc)

    print(f"Epoch [{epoch+1}/{config['num_epochs']}], Loss: {epoch_loss:.4f}, Train Acc: {train_acc:.2f}%, Val Acc: {val_acc:.2f}%")

    # Save model to Google Drive
    torch.save(model.state_dict(), model_save_path)

# Load the saved model
model.load_state_dict(torch.load(model_save_path))
model.eval()
print("Model loaded successfully.")

# Final accuracy
total_train_accuracy = sum(train_accuracies) / len(train_accuracies)
total_val_accuracy = sum(val_accuracies) / len(val_accuracies)
print(f"\nFinal Train Accuracy: {total_train_accuracy:.2f}%")
print(f"Final Validation Accuracy: {total_val_accuracy:.2f}%")

# Test dataset
test_dataset = PlantDiseaseDataset(val_paths, val_labels, transform=transform)
test_loader = DataLoader(test_dataset, batch_size=config["train_batch_size"], shuffle=False)

# Test Accuracy
test_accuracy = calculate_accuracy(model, test_loader)
print(f"Test Accuracy: {test_accuracy:.2f}%")

# Plot accuracy and loss
plt.figure(figsize=(12, 5))

# Accuracy plot
plt.subplot(1, 2, 1)
plt.plot(range(1, config["num_epochs"] + 1), train_accuracies, label="Train Accuracy")
plt.plot(range(1, config["num_epochs"] + 1), val_accuracies, label="Validation Accuracy")
plt.title("Training & Validation Accuracy")
plt.xlabel("Epoch")
plt.ylabel("Accuracy (%)")
plt.legend()

# Loss plot
plt.subplot(1, 2, 2)
plt.plot(range(1, config["num_epochs"] + 1), train_losses, label="Loss")
plt.title("Training Loss")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.legend()

plt.tight_layout()
plt.show()


In [ ]:
!python /content/drive/MyDrive/chatbot.py

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100% 44.7M/44.7M [00:00<00:00, 87.9MB/s]
/content/drive/MyDrive/chatbot.py:69: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to c